In [129]:
import dotenv
import os

import requests
import json

from datetime import datetime

import pandas as pd
import numpy as np

In [130]:
# 테스트

dotenv_file = dotenv.find_dotenv()
dotenv.load_dotenv(dotenv_file)

url = os.environ["weather_url"] + "/getUltraSrtNcst"
print(url)

now = datetime.now()
basetime = str(now.hour)+str(now.minute)

today = datetime.today().strftime('%Y%m%d')

nx = 60
ny = 127

print(today)
print(basetime)

params ={'serviceKey' : os.environ["decode_key"], 'pageNo' : '1', 'numOfRows' : '1000', 'dataType' : 'JSON', 'base_date' : today, 'base_time' : basetime, 'nx' : nx, 'ny' : ny }

res = requests.get(url, params=params)
result = json.loads(res.content)

if result['response']['header']['resultCode'] != '03' :
  items = result['response']['body']['items']['item']
  print(items)

  for i in items :
    print(i['category'], i['obsrValue'])
else :
  print(result['response']['header']['resultMsg'])

http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst
20231010
149
NO_DATA


In [131]:
location_data = pd.read_excel('./weather_api/기상청41_단기예보 조회서비스_오픈API활용가이드_격자_위경도(20230611).xlsx')
location_data.head()

,구분,행정구역코드,1단계,2단계,3단계,격자 X,격자 Y,경도(시),경도(분),경도(초),위도(시),위도(분),위도(초),경도(초/100),위도(초/100),위치업데이트
0,kor,1100000000,서울특별시,NaN,NaN,60,127,126,58,48.03,37,33,48.85,126.980008,37.563569,NaN
1,kor,1111000000,서울특별시,종로구,NaN,60,127,126,58,53.91,37,34,13.36,126.981642,37.570378,NaN
2,kor,1111051500,서울특별시,종로구,청운효자동,60,127,126,58,14.35,37,35,2.89,126.970652,37.584137,NaN
3,kor,1111053000,서울특별시,종로구,사직동,60,127,126,58,15.44,37,34,23.77,126.970956,37.573269,NaN
4,kor,1111054000,서울특별시,종로구,삼청동,60,127,126,59,2.32,37,34,56.73,126.983978,37.582425,NaN


In [132]:
url = os.environ["weather_url"] + "/getVilageFcst"

params ={'serviceKey' : os.environ["decode_key"], 'pageNo' : '1', 'numOfRows' : '1000', 'dataType' : 'JSON', 'base_date' : today, 'base_time' : basetime, 'nx' : nx, 'ny' : ny }

res = requests.get(url, params=params)
# print(res.content)
result = json.loads(res.content)
print(result)

# with open(f'{today}{basetime}_weather.json', 'w') as f :
#   json.dump(result, f)

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL_SERVICE'}, 'body': {'dataType': 'JSON', 'items': {'item': [{'baseDate': '20231010', 'baseTime': '1400', 'category': 'TMP', 'fcstDate': '20231010', 'fcstTime': '1500', 'fcstValue': '23', 'nx': 60, 'ny': 127}, {'baseDate': '20231010', 'baseTime': '1400', 'category': 'UUU', 'fcstDate': '20231010', 'fcstTime': '1500', 'fcstValue': '-1', 'nx': 60, 'ny': 127}, {'baseDate': '20231010', 'baseTime': '1400', 'category': 'VVV', 'fcstDate': '20231010', 'fcstTime': '1500', 'fcstValue': '-0.7', 'nx': 60, 'ny': 127}, {'baseDate': '20231010', 'baseTime': '1400', 'category': 'VEC', 'fcstDate': '20231010', 'fcstTime': '1500', 'fcstValue': '58', 'nx': 60, 'ny': 127}, {'baseDate': '20231010', 'baseTime': '1400', 'category': 'WSD', 'fcstDate': '20231010', 'fcstTime': '1500', 'fcstValue': '1.3', 'nx': 60, 'ny': 127}, {'baseDate': '20231010', 'baseTime': '1400', 'category': 'SKY', 'fcstDate': '20231010', 'fcstTime': '1500', 'fcstValue': '1', '

In [134]:
weather_data = result['response']['body']['items']['item']

weather_df = pd.DataFrame(weather_data)

# weather_df[weather_df['fcstTime'] == '1200']
# weather_df.iloc[:36]
weather_df

,baseDate,baseTime,category,fcstDate,fcstTime,fcstValue,nx,ny
0,20231010,1400,TMP,20231010,1500,23,60,127
1,20231010,1400,UUU,20231010,1500,-1,60,127
2,20231010,1400,VVV,20231010,1500,-0.7,60,127
3,20231010,1400,VEC,20231010,1500,58,60,127
4,20231010,1400,WSD,20231010,1500,1.3,60,127
...,...,...,...,...,...,...,...,...
695,20231010,1400,POP,20231013,0000,0,60,127
696,20231010,1400,WAV,20231013,0000,0,60,127
697,20231010,1400,PCP,20231013,0000,강수없음,60,127
698,20231010,1400,REH,20231013,0000,75,60,127


In [155]:
pivot_df = weather_df.pivot(index=['fcstDate','fcstTime'], columns='category', values='fcstValue')

weather_data = pivot_df.reset_index()


weather_data.describe()
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   fcstDate  58 non-null     object
 1   fcstTime  58 non-null     object
 2   PCP       58 non-null     object
 3   POP       58 non-null     object
 4   PTY       58 non-null     object
 5   REH       58 non-null     object
 6   SKY       58 non-null     object
 7   SNO       58 non-null     object
 8   TMN       2 non-null      object
 9   TMP       58 non-null     object
 10  TMX       2 non-null      object
 11  UUU       58 non-null     object
 12  VEC       58 non-null     object
 13  VVV       58 non-null     object
 14  WAV       58 non-null     object
 15  WSD       58 non-null     object
dtypes: object(16)
memory usage: 7.4+ KB


In [159]:
weather_data.head()

category,fcstDate,fcstTime,PCP,POP,PTY,REH,SKY,SNO,TMN,TMP,TMX,UUU,VEC,VVV,WAV,WSD
0,20231010,1500,강수없음,0,0,45,1,적설없음,NaN,23,NaN,-1,58,-0.7,0,1.3
1,20231010,1600,강수없음,0,0,45,1,적설없음,NaN,22,NaN,-0.9,63,-0.5,0,1.1
2,20231010,1700,강수없음,0,0,45,1,적설없음,NaN,22,NaN,-0.6,74,-0.2,0,0.7
3,20231010,1800,강수없음,0,0,55,1,적설없음,NaN,21,NaN,0,135,0.1,0,0.1
4,20231010,1900,강수없음,0,0,60,1,적설없음,NaN,19,NaN,-0.1,135,0.2,0,0.3


In [158]:
not_null_tmn = weather_data[weather_data['TMN'].notnull()]
not_null_tmx = weather_data[weather_data['TMX'].notnull()]

print(not_null_tmn)
print(not_null_tmx)

category  fcstDate fcstTime   PCP POP PTY REH SKY   SNO   TMN TMP  TMX   UUU  \
15        20231011     0600  강수없음  20   0  75   3  적설없음  13.0  13  NaN  -1.2   
39        20231012     0600  강수없음   0   0  80   1  적설없음  13.0  13  NaN    -1   

category  VEC  VVV WAV  WSD  
15         94  0.1   0  1.3  
39        100  0.2   0  1.1  
category  fcstDate fcstTime   PCP POP PTY REH SKY   SNO  TMN TMP   TMX  UUU  \
24        20231011     1500  강수없음  20   0  40   3  적설없음  NaN  23  23.0  0.6   
48        20231012     1500  강수없음   0   0  45   1  적설없음  NaN  22  22.0    1   

category  VEC   VVV WAV  WSD  
24        252   0.2   0  0.6  
48        320  -1.2   0  1.6  
